In [1]:
%cd ../scripts 
from address import addressToCords
import pandas as pd

/mnt/c/Users/ohanr/OneDrive/Unimelb/3rd Year/MAST30034/generic-real-estate-consulting-project-group-25/scripts


In [2]:
from geopy.geocoders import Nominatim
import re
def addressToCords(address: str):    
    geolocator = Nominatim(user_agent="addToCords")
    location = geolocator.geocode(address)
    if location is None:
        try:
            # Getting rid of unit number
            address = re.sub("^.+\/", "", address)
            location = geolocator.geocode(address)
            return [location.latitude, location.longitude]
        except AttributeError:
            try:
                address = re.sub("[0-9]+\-", "", address)
                location = geolocator.geocode(address)
                return [location.latitude, location.longitude]
            except AttributeError:
                return "Address not in DataBase"        

    return [location.latitude, location.longitude]
 

In [2]:
listing = pd.read_csv("../data/curated/cleaned_property_listing_data.csv")
listing = pd.read_csv("../data/curated/listings_w_cords.csv")

In [6]:
listing["Coordinates"] = None
listing

,Unnamed: 0,id,price,beds,baths,parking,title,address,propertyType,status,bond,Coordinates
0,0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,None
1,1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0,None
2,2,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0,None
3,3,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0,None
4,4,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...
12400,12482,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0,None
12401,12483,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0,None
12402,12484,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0,None
12403,12485,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0,None


In [3]:
#Dividing the dataframe into chunks to prevent loss of data when the Nominatium (API) server from goes down
n = 100
start = 1200
end = 1300
while end < len(listing):
    try: 
        listing["Coordinates"].iloc[start: end] = listing["address"].iloc[start:end].apply(lambda x: addressToCords(x))
        listing.to_csv("../data/curated/listings_w_cords.csv")
    except:
        print(start)
        break
    start = end 
    end += n

#1100 to 1200 ?


1200


In [4]:
listing["Coordinates"].iloc[1200: 1300] = listing["address"].iloc[1200:1300].apply(lambda x: addressToCords(x))
#listing.to_csv("../data/curated/listings_w_cords.csv")

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1805%2F50+Albert+Road%2C+South+Melbourne+VIC+3205&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))